### Imports, Reading in files & Defining functions

In [11]:
import os
import io
import regex as re
import numpy as np
from tqdm import tqdm
from termcolor import colored
from colorama import Back, Style

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = [word.upper() for word in stopwords.words('english')]
hard_coded_non_section_words = ['URL', 'UTC', 'JSTOR', 'AMERICAN', 'JOURNAL', 'SOCIOLOGY', 'ABSTRACT', 'TABLE', 'CHART',
                                'AMERICAN JOURNAL OF SOCIOLOGY', 'UNIVERSITY OF CHICAGO', 'AMERICA', 'JOURNAL OF SOCIOLOGY',
                                'REFERENCES', 'AMERICAN SOCIOLOGICAL REVIEW', 'AMERICAN SOCIOLOGICAL ASSOCIATION']

def PDFtoString(filePath, pdfFolder=None):
    
    out = io.StringIO()
    if pdfFolder is not None:
        filePath = os.path.join(pdfFolder, filePath)
    with open(filePath, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, out, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for indx, page in enumerate(PDFPage.create_pages(doc)):
            #if indx != 0:
            interpreter.process_page(page)
            
    return out.getvalue() 

def printMetaInfo(convertedStrings, pdfPaths, journal = 'AJS'):
    
    # Print the regex matches for access date, abstract, table/chart, and references
    for indx, string in enumerate(convertedStrings):
        print(indx, colored(pdfPaths[indx], 'red'))
        if re.findall(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', string):
            print('\t', colored(re.findall(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', string)[0], 'blue'))
        print('\t', colored('Abstract match:', 'magenta'), colored(re.search(r'ABSTRACT', string), 'magenta'))
        
        # Chart, table, or figure matches
        matches = re.finditer(r'CHART', string)
        for match in matches:
            print('\t', colored('Chart match:', 'green'), colored(match, 'green'))
        matches = re.finditer(r'TABLE', string)
        for match in matches:
            print('\t', colored('Table match:', 'green'), colored(match, 'green'))
        matches = re.finditer(r'FIG|FIGURE|Fig', string)
        for match in matches:
            print('\t', colored('Figure match:', 'green'), colored(match, 'green'))  
            
        print('\t', colored('Reference match:', 'magenta'), colored(re.search(r'REFERENCES', string), 'magenta'))
        
        # This finds the title words which aren't stop words & aren't digits and upper cases them all
        if journal == 'AJS':
            titles = [re.findall(r'(?<=AJS_\d{4}_\d{1,3}_\d{1,2}_).*(?=.pdf)', file_name)[0] for file_name in pdfPaths]
        elif journal == 'ASR':
            titles = [re.findall(r'(?<=ASR_\d{4}_\d{1,3}_\d{1,2}_).*(?=.pdf)', file_name)[0] for file_name in pdfPaths]
        title_words = [i for i in titles[indx].upper().split() if i not in stop_words and not i.isdigit()]
        
        # Then adds them to stop words & other hard-coded regularly occuring words
        non_section_words = hard_coded_non_section_words + title_words + stop_words 
        non_section_words += [title.upper() for title in titles] + [title.strip('The ').upper() for title in titles]
        
        matches = re.finditer(r'[A-Z]{4,}(\s+?[A-Z]{2,}){0,}', string)
        sections = dict()
        
        for match in matches:
            if match.group() not in non_section_words and match not in sections:
                sections[match.group()] = match.span()
        for section in sections:
            print('\t', colored('Section match:', 'blue'), colored([section, sections[section]], 'blue'))

def createOutputStrings(convertedStrings, folder = None, journal = 'AJS'):
    out = list() #Output strings go here, hopefully one out per pdf s.t. len(out) == len(convertedStrings)
    for indx, string in enumerate(convertedStrings):
        # Define meta-information
        outString = '-------------\n||Meta-info||\n-------------\n'

        # The second condition is more restrictive, so check that that's not None.
        header = article = None
        if re.search(r'extend access to\n(.)+?(?=C)(.)+?(?=\w{2,})', string) is not None:
            header = string[:re.search(r'extend access to\n(.)+?(?=C)', string).end()].strip()
            if re.search(r' Your use of the JSTOR', header) is not None:
                header = header[:re.search(r' Your use of the JSTOR', header).span()[0]]
            article = string[re.search(r'extend access to\n(.)+?(?=C)(.)+?(?=\w{2,})', string).end():]
            outString += re.sub(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', '', header)

        # Add access date
        outString += '\n\n' + re.findall(r'Accessed: \d{2}-\d{2}-\d{4} \d{2}:\d{2} UTC', string)[0] + '\n'

        # Add abstract info
        abstract = re.search(r'ABSTRACT', string)
        if abstract is not None:
            outString += 'Abstract match at {}'.format(abstract.span()) + '\n'
        elif abstract is None:
            outString += 'No abstract found.\n'

        # Add chart info
        chart_matches = re.finditer(r'CHART', string)    
        if chart_matches is not None:
            for match in chart_matches:
                outString += 'Chart match at {}'.format(match.span()) + '\n'

        # Add table info 
        table_matches = re.finditer(r'TABLE', string)
        if table_matches is not None:
            for match in table_matches:
                outString += 'Chart match at {}'.format(match.span()) + '\n'
                
        # Add figure info
        figure_matches = re.finditer(r'FIG|FIGURE|Fig', string)
        if figure_matches is not None:
            for match in figure_matches:
                outString += 'Figure match at {}'.format(match.span()) + '\n'

        # Add references info
        references = re.search(r'REFERENCES', string)
        if references is not None:
            outString += 'Reference match at {}'.format(references.span()) + '\n'
        elif references is None:
            outString += 'No reference section found.\n'

        # Add section info 

        ### First, look for the title words and tokenize them.
        if journal == 'AJS':
            titles = [re.findall(r'(?<=AJS_\d{4}_\d{1,3}_\d{1,2}_).*(?=.pdf)', file_name)[0] for file_name in folder]
        elif journal == 'ASR':
            titles = [re.findall(r'(?<=ASR_\d{4}_\d{1,3}_\d{1,2}_).*(?=.pdf)', file_name)[0] for file_name in folder]
        title_words = [i for i in titles[indx].upper().split() if i not in stop_words and not i.isdigit()]

        ### Then add them to stop words & other hard-coded regularly occuring words
        non_section_words = hard_coded_non_section_words + title_words + stop_words
        matches = re.finditer(r'[A-Z]{4,}(\s+?[A-Z0-9]{2,}){0,}', string)

        if matches is not None:
            sections = dict()
            for match in matches:
                if match.group() not in non_section_words and match not in sections:
                    sections[match.group()] = match.span()
        for section in sections:
            outString += 'Section header "{}" found at {}'.format(section, sections[section]) + '\n'

        # Add in the article itself
        outString += '\n-----------\n||Article||\n-----------\n'
        if article is not None:
            outString += article
        elif article is None: #If regex wasn't able to split it according to the JSTOR access message just put the full article
            outString += string
        # Append the newly made string to the list. Again, there should be 1 per pdf
        out.append(outString)
    return out
    
def writeOut(out, pdfFolder = None, outFolder = None):
    for indx, file in tqdm(enumerate(out)):
        writeFilePath = 'corpus/{}/{}.txt'.format(outFolder, pdfFolder[indx][:-4])
        with open(writeFilePath, 'w') as f:
            f.write(file)
    print('done!')

def highlight(pattern, text, print_output=True):
    output = text
    lookforward = 0
    for match in pattern.finditer(text):
        start, end = match.start() + lookforward, match.end() + lookforward
        output = output[:start] + Back.YELLOW + Style.BRIGHT + output[start:end] + Style.RESET_ALL + output[end:]
        lookforward = len(output) - len(text)  

    if print_output:
        print(output)
    else:
        return output

## American Journal of Sociology articles

In [2]:
%%time
# AJS articles - split into 3 periods

pre1946 = '/Users/Praveens/Desktop/ishan/Language-of-Science/articles/AJS pdf files/pre1946/'
pre1946pdfs = os.listdir(pre1946) # list of all the pdf files 
pre1946pdfs.sort() # sort by year (and title)

l946to1966 = '/Users/Praveens/Desktop/ishan/Language-of-Science/articles/AJS pdf files/1946to1966/'
l946to1966pdfs = os.listdir(l946to1966) # list of all the pdf files
l946to1966pdfs.sort()

post1971 = '/Users/Praveens/Desktop/ishan/Language-of-Science/articles/AJS pdf files/post1971/'
post1971pdfs = os.listdir(post1971) # list of all the pdf files
post1971pdfs.sort()

# Convert the articles to strings... this is the time-consuming step
convertedStrings_pre1946 = [PDFtoString(os.path.join(pre1946, file)) for file in tqdm(pre1946pdfs) if file[-4:] == '.pdf']
convertedStrings_1946to1966 = [PDFtoString(os.path.join(l946to1966, file)) for file in tqdm(l946to1966pdfs) if file[-4:] == '.pdf']
convertedStrings_post1971 = [PDFtoString(os.path.join(post1971, file)) for file in tqdm(post1971pdfs) if file[-4:] == '.pdf']

100%|██████████| 68/68 [04:02<00:00,  3.57s/it]

CPU times: user 5min 47s, sys: 8.96 s, total: 5min 56s
Wall time: 6min 45s


### String pre-processing

In [3]:
# Remove the double lines / extra space characters & footer download / use notice
convertedStrings_pre1946 = [' '.join(re.split('\n\n+', string)) for string in convertedStrings_pre1946]
pattern = re.compile(r'This content downloaded from (.)+\n(.)+\x0c')
#highlight(pattern, convertedStrings_pre1946[4])
convertedStrings_pre1946 = [' '.join(re.split(r'This content downloaded from (.)+\n(.)+\x0c', string)) for string in convertedStrings_pre1946]
printMetaInfo(convertedStrings_pre1946, pdfPaths = pre1946pdfs)

0 AJS_1896_1_4_Anti-Monopoly Legislation in the US.pdf
	 Accessed: 08-05-2016 22:57 UTC
	 Abstract match: None
	 Reference match: None
	 Section match: ['ANTI', (32901, 32905)]
	 Section match: ['MONOPOLY LEGISLATION IN THE UNITED  STATES', (897, 939)]
	 Section match: ['RECENT', (942, 948)]
	 Section match: ['MONOPOLY LEGISLA TION', (32907, 32928)]
	 Section match: ['XXIV', (7166, 7170)]
	 Section match: ['MONOPOLY LEGISLATION', (9796, 9816)]
	 Section match: ['MONOPOL', (18830, 18837)]
	 Section match: ['LEGISLA TION', (28510, 28522)]
	 Section match: ['XIII', (16283, 16287)]
	 Section match: ['XVII', (16416, 16420)]
	 Section match: ['HALLE', (21144, 21149)]
	 Section match: ['MONVOPOL', (28499, 28507)]
	 Section match: ['FORREST', (33722, 33729)]
1 AJS_1896_1_4_Business Men and Social Theorists.pdf
	 Accessed: 08-05-2016 22:26 UTC
	 Abstract match: None
	 Reference match: None
	 Section match: ['AMERICAN  JOURNAL OF SOCIOLOGY  VOLUME', (885, 923)]
	 Section match: ['JANUARY', (926,

	 Section match: ['AMERICAN  JOURNAL OF SOCIOLOGY  VOLUME XXVI JANUARY', (885, 936)]
	 Section match: ['NUMBER', (942, 948)]
	 Section match: ['SOCIOLOGY AND THE SOCIAL SCIENCES  ROBERT', (952, 993)]
	 Section match: ['PARK', (997, 1001)]
	 Section match: ['SOCIOLOGY AND', (1029, 1042)]
	 Section match: ['SCIENTIFIC', (1045, 1055)]
	 Section match: ['HISTORY', (40802, 40809)]
	 Section match: ['XVII', (10093, 10097)]
	 Section match: ['HISTORICAL AND SOCIOLOGICAL FACTS', (13638, 13671)]
	 Section match: ['HUMAN NATURE AND LAW', (30790, 30810)]
	 Section match: ['NATURAL HISTORY', (40811, 40826)]
	 Section match: ['XLIV', (43772, 43776)]
30 AJS_1921_26_4_The Comparative Role of the Group Concept.pdf
	 Accessed: 10-05-2016 17:14 UTC
	 Abstract match: None
	 Reference match: None
	 Section match: ['COMPARATIVE ROLE OF THE GROUP CONCEPT  IN WARD', (963, 1009)]
	 Section match: ['DYNAMIC SOCIOLOGY AND CONTEM', (1012, 1040)]
	 Section match: ['PORARY AMERICAN SOCIOLOGY  WALTER', (1043, 1076)

	 Section match: ['AMERICAN  JOURNAL OF SOCIOLOGY  VOLUME XLI JANUARY', (931, 981)]
	 Section match: ['NUMBER', (987, 993)]
	 Section match: ['SOCIAL MOBILITY AND SOCIAL DISTANCE AMONG  HUNGARIAN IMMIGRANTS IN DETROIT', (997, 1071)]
	 Section match: ['ERDMANN DOANE BEYNON  ABSTRACT', (1074, 1104)]
	 Section match: ['HUNGARIAN IMMIGRANTS IN DETROIT', (26656, 26687)]
	 Section match: ['UNIVERSITY OF MICHIGAN', (31668, 31690)]
	 Section match: ['XIII', (32042, 32046)]
52 AJS_1936_41_4_The Length of Time Required for the Stabilization of a Population.pdf
	 Accessed: 03-06-2016 20:01 UTC
	 Abstract match: <regex.Match object; span=(1007, 1015), match='ABSTRACT'>
	 Table match: <regex.Match object; span=(9343, 9348), match='TABLE'>
	 Table match: <regex.Match object; span=(11074, 11079), match='TABLE'>
	 Table match: <regex.Match object; span=(15887, 15892), match='TABLE'>
	 Table match: <regex.Match object; span=(15909, 15914), match='TABLE'>
	 Table match: <regex.Match object; span=(18037,

In [4]:
convertedStrings_1946to1966 = [' '.join(re.split('\n\n+', string)) for string in convertedStrings_1946to1966]
pattern = re.compile(r'[A-Z]{4,}(\s+?[A-Z0-9]{2,}){0,}')
highlight(pattern, convertedStrings_1946to1966[10])
convertedStrings_1946to1966 = [' '.join(re.split(r'This content downloaded from (.)+\n(.)+\x0c', string)) for string in convertedStrings_1946to1966]
printMetaInfo(convertedStrings_1946to1966, pdfPaths = l946to1966pdfs)

      Job-Seeking and the Readjustment Allowance for Veterans
Author(s): Henry J. Meyer and  Erwin O. Smigel
Source: American Journal of Sociology, Vol. 56, No. 4 (Jan., 1951), pp. 341-347
Published by: The University of Chicago Press
Stable URL: http://www.jstor.org/stable/2771697
Accessed: 06-06-2016 18:32 UTC Your use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at http://about.jstor.org/terms JSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of content in a trusted digital archive. We use information technology and tools to increase productivity and facilitate new forms of scholarship. For more information about JSTOR, please contact support@jstor.org. The University of Chicago Press is collaborating with JSTOR to digitize, preserve and extend access to
American Journal of Sociology This content downloaded from 170.140.26.180 on Mon, 06 Jun 2016 18:32:33 UTC
All u

	 Section match: ['TYPES OF INTEGRATION AND THEIR MEASUREMENT  WERNER', (895, 945)]
	 Section match: ['LANDECKER', (949, 958)]
	 Section match: ['CULTURAL INTEGRATION', (7244, 7264)]
	 Section match: ['NORMATIVE INTEGRATION', (13672, 13693)]
	 Section match: ['XLVII', (15568, 15573)]
	 Section match: ['COMMUNICATIVE INTEGRATION', (18328, 18353)]
	 Section match: ['XLIX', (21928, 21932)]
	 Section match: ['XIII', (22137, 22141)]
	 Section match: ['FUNCTIONAL INTEGRATION', (27705, 27727)]
	 Section match: ['UNIVERSITY OF MICHIGAN', (37174, 37196)]
13 AJS_1951_56_4_Witch Beliefs and Social Structure.pdf
	 Accessed: 06-06-2016 18:27 UTC
	 Abstract match: <regex.Match object; span=(1003, 1011), match='ABSTRACT'>
	 Reference match: None
	 Section match: ['JANUARY', (921, 928)]
	 Section match: ['MONICA HUNTER WILSON  ABSTRACT', (981, 1011)]
	 Section match: ['WITCH RELIEFS AND SOCIAL STRUCTURE', (8099, 8133)]
	 Section match: ['RHODES UNIVERSITY COLLEGE', (29429, 29454)]
14 AJS_1956_61_4_Ano

	 Reference match: <regex.Match object; span=(306, 316), match='REFERENCES'>
	 Section match: ['DRUGGISTS', (30205, 30214)]
	 Section match: ['PROBLEMS OF', (1161, 1172)]
	 Section match: ['MARGINAL OCCUPATION  THELMA HERMAN MCCORMACK  ABSTRACT', (1176, 1230)]
	 Section match: ['XXIV', (20540, 20544)]
	 Section match: ['COMPARISONS OF PRESTIGE RATINGS AND RANKS GIVEN BY  PHARMACY STUDENTS', (24028, 24097)]
	 Section match: ['NATIONAL SAMPLE AND  SUBGROUPS WITHIN NATIONAL SAMPLE', (24099, 24152)]
	 Section match: ['OCCUPATIONS CISTS TOTAL', (24156, 24179)]
	 Section match: ['PHARMA', (24203, 24209)]
	 Section match: ['NATIONAL SAMPLE', (29610, 29625)]
	 Section match: ['XIII', (25335, 25339)]
	 Section match: ['PRESTIGE RATINGS AND RANKS GIVEN TO SELECTED GROUP OF OCCU', (27658, 27716)]
	 Section match: ['PATIONS BY PHARMACY STUDENTS AND', (27719, 27751)]
	 Section match: ['AVERAGE PRESTIGE SCORES FOR MAJOR CATE', (29511, 29549)]
	 Section match: ['GORIES OF OCCUPATIONS GIVEN BY PHARMA'

	 Section match: ['NATHAN KEYFITZ  ABSTRACT', (915, 939)]
	 Section match: ['XIII', (18021, 18025)]
	 Section match: ['UNIVERSITY OF TORONTO', (30086, 30107)]
29 AJS_1961_66_4_Theory, Measurement, and Replication in the Social Sciences.pdf
	 Accessed: 06-06-2016 18:53 UTC
	 Abstract match: <regex.Match object; span=(994, 1002), match='ABSTRACT'>
	 Reference match: None
	 Section match: ['THEORY', (22631, 22637)]
	 Section match: ['MEASUREMENT', (22639, 22650)]
	 Section match: ['REPLICATION  IN THE SOCIAL SCIENCES', (936, 971)]
	 Section match: ['BLALOCK', (980, 987)]
	 Section match: ['XXVI', (4402, 4406)]
	 Section match: ['MEASUREMENT OF MASS', (6029, 6048)]
	 Section match: ['XLIX', (6988, 6992)]
	 Section match: ['MEASUREMENT OF POWER', (12843, 12863)]
	 Section match: ['MEASUREMENT OF DISCRIMINATION', (18396, 18425)]
	 Section match: ['UNIVERSITY OF MICHIGAN', (25726, 25748)]
30 AJS_1966_71_4_Group Differences in Perception A Study of Community Beliefs and Feelings About Tubercul

In [5]:
convertedStrings_post1971 = [' '.join(re.split('\n\n+', string)) for string in convertedStrings_post1971]
convertedStrings_post1971 = [' '.join(re.split(r'This content downloaded from (.)+\n(.)+\x0c', string)) for string in convertedStrings_post1971]
pattern = re.compile(r'[A-Z]{4,}(\s+?[A-Z0-9]{2,}){0,}')
highlight(pattern, convertedStrings_post1971[2])
printMetaInfo(convertedStrings_post1971, pdfPaths = post1971pdfs)

      An Empirical Study of Military-Industrial Linkages
Author(s): Stanley Lieberson
Source: American Journal of Sociology, Vol. 76, No. 4 (Jan., 1971), pp. 562-584
Published by: The University of Chicago Press
Stable URL: http://www.jstor.org/stable/2776429
Accessed: 08-06-2016 02:59 UTC Your use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at http://about.jstor.org/terms JSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide range of content in a trusted digital archive. We use information technology and tools to increase productivity and facilitate new forms of scholarship. For more information about JSTOR, please contact support@jstor.org. The University of Chicago Press is collaborating with JSTOR to digitize, preserve and extend access to
American Journal of Sociology  C    An Empirical Study of Military-Industrial  Linkages'  Stanley Lieberson  University of Washington  

	 Section match: ['DETERMINANTS OF FEMALE RETIREMENT', (12151, 12184)]
	 Section match: ['METHODOLOGY', (21681, 21692)]
	 Section match: ['MGLS', (56229, 56233)]
	 Section match: ['RESULTS', (41033, 41040)]
	 Section match: ['MEANS AND STANDARD DEVIATIONS OF AGE', (42786, 42822)]
	 Section match: ['SPECIFI', (42823, 42830)]
	 Section match: ['LABOR FORCE PARTICIPATION OF  WOMEN AND OF RESIDUAL RETIREMENT RATES', (42832, 42900)]
	 Section match: ['YEAR AND BY REGION  MEAN FEMALE  PARTICIPATION RATES', (42905, 42957)]
	 Section match: ['MEAN RETIREMENT RATES', (43017, 43038)]
	 Section match: ['NOTE', (43615, 43619)]
	 Section match: ['CONCLUSIONS', (56872, 56883)]
23 AJS_1986_91_4_Economic Segmentation and Politics.pdf
	 Accessed: 08-06-2016 03:51 UTC
	 Abstract match: None
	 Reference match: <regex.Match object; span=(62106, 62116), match='REFERENCES'>
	 Section match: ['GENERAL BACKGROUND', (1873, 1891)]
	 Section match: ['SPECIFIC THEORETICAL CONCERNS', (6406, 6435)]
	 Section match:

	 Reference match: <regex.Match object; span=(86188, 86198), match='REFERENCES'>
	 Section match: ['INTRODUCTION', (1846, 1858)]
	 Section match: ['GENESIS AND DEVELOPMENT OF TWO ECOLOGIES', (11135, 11175)]
	 Section match: ['LCRC', (36683, 36687)]
	 Section match: ['FIGURE AND TROPE IN THE LANGUAGE OF ECOLOGY', (38169, 38212)]
	 Section match: ['ASSOCIATIVE RELATIONS OF', (44709, 44733)]
	 Section match: ['HUMAN ECOLOGY', (44735, 44748)]
	 Section match: ['BOUNDARY WORK IN DISCIPLINES', (51883, 51911)]
	 Section match: ['SUBDISCIPLINES', (51913, 51927)]
	 Section match: ['SCHOOLS', (51934, 51941)]
	 Section match: ['DISSOLUTION OF CLASSICAL HUMAN ECOLOGY', (67196, 67234)]
	 Section match: ['CONCLUSION', (80436, 80446)]
35 AJS_1996_101_4_Markets and Inequality in Transitional Economies-Toward Testable Theories.pdf
	 Accessed: 08-06-2016 04:18 UTC
	 Abstract match: None
	 Table match: <regex.Match object; span=(25339, 25344), match='TABLE'>
	 Reference match: <regex.Match object; span=(

	 Abstract match: None
	 Table match: <regex.Match object; span=(24705, 24710), match='TABLE'>
	 Table match: <regex.Match object; span=(63366, 63371), match='TABLE'>
	 Table match: <regex.Match object; span=(68817, 68822), match='TABLE'>
	 Table match: <regex.Match object; span=(77891, 77896), match='TABLE'>
	 Table match: <regex.Match object; span=(86920, 86925), match='TABLE'>
	 Table match: <regex.Match object; span=(89310, 89315), match='TABLE'>
	 Table match: <regex.Match object; span=(119771, 119776), match='TABLE'>
	 Table match: <regex.Match object; span=(126810, 126815), match='TABLE'>
	 Reference match: <regex.Match object; span=(138805, 138815), match='REFERENCES'>
	 Section match: ['INTRODUCTION', (2104, 2116)]
	 Section match: ['STANFORD PROJECT ON EMERGING COMPANIES', (11759, 11797)]
	 Section match: ['SPEC', (132842, 132846)]
	 Section match: ['DATA COLLECTION', (13249, 13264)]
	 Section match: ['FOUNDERS', (16153, 16161)]
	 Section match: ['ORGANIZATIONAL MODELS', (161

### Writing the files out to .txt files

In [12]:
out_pre1946 = createOutputStrings(convertedStrings_pre1946, folder = pre1946pdfs)
out_1946to1966 = createOutputStrings(convertedStrings_1946to1966, folder = l946to1966pdfs)
out_post1971 = createOutputStrings(convertedStrings_post1971, folder = post1971pdfs)

writeOut(out_pre1946, pdfFolder = pre1946pdfs, outFolder = 'AJS_pre1946')
writeOut(out_1946to1966, pdfFolder = l946to1966pdfs, outFolder = 'AJS_1946to1966')
writeOut(out_post1971, pdfFolder = post1971pdfs, outFolder = 'AJS_post1971')

63it [00:00, 4426.97it/s]
37it [00:00, 4184.13it/s]
68it [00:00, 2327.09it/s]

done!
done!
done!


## American Sociological Review articles

In [7]:
ASRpre1946 = '/Users/Praveens/Desktop/ishan/Language-of-Science/articles/ASR pdf files/pre1946/'
ASRpre1946pdfs = os.listdir(ASRpre1946) # list of all the pdf files 
ASRpre1946pdfs.sort() # sort by year (and title)

ASRpost1946 = '/Users/Praveens/Desktop/ishan/Language-of-Science/articles/ASR pdf files/post1946/'
ASRpost1946pdfs = os.listdir(ASRpost1946) # list of all the pdf files
ASRpost1946pdfs.sort()

# Convert the articles to strings... this is the time-consuming step
convertedStrings_ASRpre1946 = [PDFtoString(os.path.join(ASRpre1946, file)) for file in tqdm(ASRpre1946pdfs)]
convertedStrings_ASRpost1946 = [PDFtoString(os.path.join(ASRpost1946, file)) for file in tqdm(ASRpost1946pdfs)]

100%|██████████| 121/121 [06:13<00:00,  3.08s/it]


In [8]:
# Remove the double lines / extra space characters & footer download / use notice
convertedStrings_ASRpre1946 = [' '.join(re.split('\n\n+', string)) for string in convertedStrings_ASRpre1946]
convertedStrings_ASRpre1946 = [' '.join(re.split(r'This content downloaded from (.)+\n(.)+\x0c', string)) for string in convertedStrings_ASRpre1946]
printMetaInfo(convertedStrings_ASRpre1946, pdfPaths = ASRpre1946pdfs, journal = 'ASR')

0 ASR_1936_1_1_A Critical Study of the Criterion of Internal Consistency in Personality Scale Construction.pdf
	 Accessed: 14-06-2016 20:15 UTC
	 Abstract match: None
	 Chart match: <regex.Match object; span=(10135, 10140), match='CHART'>
	 Table match: <regex.Match object; span=(6174, 6179), match='TABLE'>
	 Table match: <regex.Match object; span=(8183, 8188), match='TABLE'>
	 Table match: <regex.Match object; span=(10159, 10164), match='TABLE'>
	 Table match: <regex.Match object; span=(10201, 10206), match='TABLE'>
	 Reference match: <regex.Match object; span=(332, 342), match='REFERENCES'>
	 Section match: ['CRITICAL STUDY OF THE CRITERION OF\n INTERNAL CONSISTENCY IN PERSONALITY  SCALE CONSTRUCTION  RAYMOND', (1192, 1292)]
	 Section match: ['SLETTO', (1296, 1302)]
	 Section match: ['FUTURE LOOKS VERY BLACK', (6238, 6261)]
	 Section match: ['EASY TO HAVE', (3474, 3486)]
	 Section match: ['GOOD TIME AT', (3489, 3501)]
	 Section match: ['PARTY', (3504, 3509)]
	 Section match: ['MEMBER

In [9]:
convertedStrings_ASRpost1946 = [' '.join(re.split('\n\n+', string)) for string in convertedStrings_ASRpost1946]
convertedStrings_ASRpost1946 = [' '.join(re.split(r'This content downloaded from (.)+\n(.)+\x0c', string)) for string in convertedStrings_ASRpost1946]
printMetaInfo(convertedStrings_ASRpost1946, pdfPaths = ASRpost1946pdfs, journal = 'ASR')

0 ASR_1946_11_1_A Note on Consistency in Questionnaire Responses.pdf
	 Accessed: 16-06-2016 04:45 UTC
	 Abstract match: None
	 Reference match: None
	 Section match: ['CONSISTENCY IN QUESTIONNAIRE RESPONSES', (8622, 8660)]
	 Section match: ['CONCLUSIONS', (1757, 1768)]
	 Section match: ['NOTE ON CONSISTENCY IN QUESTIONNAIRE  RESPONSES  JOHN', (3079, 3132)]
	 Section match: ['CUBER AND JOHN', (3136, 3150)]
	 Section match: ['GERBERICH', (3154, 3163)]
	 Section match: ['SOCIOLOGISTS', (3192, 3204)]
	 Section match: ['PROBLEM', (4257, 4264)]
	 Section match: ['ADMINISTRATION OF THE QUESTIONNAIRES', (4570, 4606)]
	 Section match: ['PREPARATION OF THE QUESTIONNAIRES', (5338, 5371)]
	 Section match: ['RESULTS', (6905, 6912)]
	 Section match: ['IMPLICATIONS FOR QUESTIONNAIRE  RESEARCH', (9574, 9614)]
	 Section match: ['XXXVIII', (11107, 11114)]
1 ASR_1946_11_1_Comparative Health Factors Among the States.pdf
	 Accessed: 16-06-2016 04:48 UTC
	 Abstract match: None
	 Table match: <regex.Match ob

	 Section match: ['DISCUSSION  REINHARD BENDIX', (937, 964)]
17 ASR_1951_16_1_Identification as the Basis for a Theory of Motivation.pdf
	 Accessed: 16-06-2016 05:00 UTC
	 Abstract match: None
	 Reference match: <regex.Match object; span=(294, 304), match='REFERENCES'>
	 Section match: ['IDENTIFICATION AS THE BASIS FOR', (1152, 1183)]
	 Section match: ['THEORY  OF MOTIVATION', (1186, 1207)]
	 Section match: ['NELSON', (1210, 1216)]
	 Section match: ['FOOTE', (1220, 1225)]
	 Section match: ['THEORY OF MOTIVATION', (31584, 31604)]
18 ASR_1951_16_1_Learning Theory and Socialization.pdf
	 Accessed: 16-06-2016 05:02 UTC
	 Abstract match: None
	 Reference match: <regex.Match object; span=(271, 281), match='REFERENCES'>
	 Section match: ['DORRIAN APPLE', (1165, 1178)]
	 Section match: ['FREQUENTLY', (1199, 1209)]
	 Section match: ['SOCIAL PSYCHOLOGY OF MARRIAGE', (20207, 20236)]
	 Section match: ['ERNEST', (20239, 20245)]
	 Section match: ['MOWRER', (20249, 20255)]
	 Section match: ['HARRIET 

	 Section match: ['MINORITY COURSE', (15358, 15373)]
	 Section match: ['MORTON', (1144, 1150)]
	 Section match: ['KING', (1154, 1158)]
35 ASR_1956_21_1_Trends in Residential Segregation of Nonwhites in American Cities, 1940-1950.pdf
	 Accessed: 16-06-2016 05:14 UTC
	 Abstract match: None
	 Table match: <regex.Match object; span=(9902, 9907), match='TABLE'>
	 Figure match: <regex.Match object; span=(8705, 8708), match='Fig'>
	 Figure match: <regex.Match object; span=(13238, 13241), match='FIG'>
	 Figure match: <regex.Match object; span=(14210, 14213), match='Fig'>
	 Reference match: <regex.Match object; span=(318, 328), match='REFERENCES'>
	 Section match: ['TRENDS IN RESIDENTIAL SEGREGATION OF NONWHITES', (14081, 14127)]
	 Section match: ['TRENDS IN RESIDENTIAL SEGREGATION OF NONWHITES\n IN AMERICAN CITIES', (2949, 3015)]
	 Section match: ['DONALD', (3028, 3034)]
	 Section match: ['COWGILL', (3038, 3045)]
	 Section match: ['URING', (3070, 3075)]
	 Section match: ['GENERAL PATTERN IN', 

	 Section match: ['RANCOROUS CONFLICT IN COMMUNITY POLITICS  WILLIAM', (1140, 1189)]
	 Section match: ['GAMSON', (1193, 1199)]
	 Section match: ['RANCOROUS CONFLICT', (38438, 38456)]
	 Section match: ['STUDY DESIGN', (17848, 17860)]
	 Section match: ['RESULTS', (35375, 35382)]
	 Section match: ['RANCOROUS CONFLICT AND  POLITICAL INSTABILITY', (36307, 36352)]
	 Section match: ['RANCOROUS CONFLICT AND  STRUCTURAL CONDUCIVENESS', (36484, 36532)]
	 Section match: ['RANCOROUS CONFLICT AND CLEAVAGE', (38470, 38501)]
	 Section match: ['DISCUSSION', (42256, 42266)]
	 Section match: ['AMERICAN SOCIOLOGICAL REVIEW  TABLE', (42499, 42534)]
	 Section match: ['RANCOROUS CONFLICT AND POLITICAL\n INSTABILITY CONTROLLED FOR SIZE OF', (42538, 42606)]
	 Section match: ['DIVISION OF LABOR', (48369, 48386)]
	 Section match: ['TECHNOLOGY', (48388, 48398)]
	 Section match: ['ORGANIZATION OF PRODUCTION IN TWELVE\n COUNTRIES', (48409, 48456)]
	 Section match: ['JACK', (48460, 48464)]
	 Section match: ['GIBBS 

	 Section match: ['RESULTS', (34574, 34581)]
	 Section match: ['DISCUSSION', (45482, 45492)]
	 Section match: ['DISABILITY AS PROCESS', (55578, 55599)]
	 Section match: ['DISABILITY AND DEVIANCE', (56927, 56950)]
	 Section match: ['NORMATIVE ADAPTATIONS\n OF ROLE BEHAVIOR', (56952, 56991)]
	 Section match: ['LAWRENCE', (56995, 57003)]
	 Section match: ['HABER', (57007, 57012)]
	 Section match: ['RICHARD', (57016, 57023)]
	 Section match: ['SMITH', (57027, 57032)]
61 ASR_1971_36_1_Some Social Implications of High Density Housing.pdf
	 Accessed: 21-06-2016 19:19 UTC
	 Abstract match: None
	 Figure match: <regex.Match object; span=(19845, 19848), match='Fig'>
	 Figure match: <regex.Match object; span=(24654, 24657), match='Fig'>
	 Figure match: <regex.Match object; span=(24697, 24700), match='Fig'>
	 Figure match: <regex.Match object; span=(30740, 30743), match='Fig'>
	 Figure match: <regex.Match object; span=(32564, 32567), match='Fig'>
	 Figure match: <regex.Match object; span=(40631, 4

	 Section match: ['WORKING WIVES', (54849, 54862)]
	 Section match: ['LINDA', (925, 930)]
	 Section match: ['WAITE', (934, 939)]
	 Section match: ['INTRODUCTION', (2496, 2508)]
	 Section match: ['PROBLEM', (2514, 2521)]
	 Section match: ['MAJOR HYPOTHESES', (5552, 5568)]
	 Section match: ['METHOD AND DATA', (9893, 9908)]
	 Section match: ['MODEL AND METHODOLOGY', (10036, 10057)]
	 Section match: ['ECLF', (19312, 19316)]
	 Section match: ['RESULTS', (24095, 24102)]
	 Section match: ['SUMMARY', (54234, 54241)]
	 Section match: ['DETERMINANTS OF ADMINISTRATIVE CONTROL', (60538, 60576)]
	 Section match: ['TEST OF', (60581, 60588)]
	 Section match: ['THEORY WITH JAPANESE FACTORIES', (60591, 60621)]
	 Section match: ['PHELPS TRACY', (60624, 60636)]
	 Section match: ['KOYA AZUMI', (60663, 60673)]
73 ASR_1981_46_1_Change and Stability in Educational Stratification.pdf
	 Accessed: 21-06-2016 19:50 UTC
	 Abstract match: None
	 Figure match: <regex.Match object; span=(44710, 44713), match='Fig'>


	 Section match: ['ROBERT', (65444, 65450)]
	 Section match: ['MARE', (65454, 65458)]
	 Section match: ['SCHOOLING', (5934, 5943)]
	 Section match: ['MARRIAGE', (5945, 5953)]
	 Section match: ['ASSORTATIVE MATING', (5960, 5978)]
	 Section match: ['CHANGES IN EDUCATIONAL  ASSORTATIVE MATING', (8873, 8915)]
	 Section match: ['DATA AND METHODS', (16222, 16238)]
	 Section match: ['PUMS', (22823, 22827)]
	 Section match: ['RESULTS', (45710, 45717)]
	 Section match: ['TRENDS IN SCHOOLING AND  EDUCATIONAL ASSORTATIVE MATING', (28472, 28527)]
	 Section match: ['TIMING OF MARRIAGE', (45723, 45741)]
	 Section match: ['EDUCATIONAL ASSORTATIVE  MATING', (45793, 45824)]
	 Section match: ['SUMMARY AND CONCLUSION', (59064, 59086)]
90 ASR_1991_56_1_Group Differences in Economic Opportunity and the Timing of Marriage-Blacks and Whites in the Rural South, 1910.pdf
	 Accessed: 21-06-2016 20:37 UTC
	 Abstract match: None
	 Reference match: <regex.Match object; span=(55117, 55127), match='REFERENCES'>
	 Se

	 Section match: ['KEIRETSU NETWORKS AND  CORPORATE PERFORMANCE IN JAPAN', (976, 1029)]
	 Section match: ['BACKGROUND', (5790, 5800)]
	 Section match: ['KEIRETSU NETWORKS AND CORPORATE PERFORMANCE', (82747, 82790)]
	 Section match: ['METHODS', (30170, 30177)]
	 Section match: ['NEEDS', (41814, 41819)]
	 Section match: ['REGRESSION ANALYSES', (47252, 47271)]
	 Section match: ['TSCSREG', (55748, 55755)]
	 Section match: ['CONCLUSIONS', (76207, 76218)]
	 Section match: ['NEEDS MT', (86839, 86847)]
102 ASR_1996_61_1_On Realization in Everyday Life-The Forecasting of Bad News as a Social Relation.pdf
	 Accessed: 22-06-2016 19:06 UTC
	 Abstract match: None
	 Reference match: <regex.Match object; span=(93506, 93516), match='REFERENCES'>
	 Section match: ['EVERYDAY LIFE', (102225, 102238)]
	 Section match: ['FORECASTING OF BAD NEWS AS', (1003, 1029)]
	 Section match: ['SOCIAL RELATION', (1032, 1047)]
	 Section match: ['LITERATURE ON BAD NEWS', (9207, 9229)]
	 Section match: ['FORECASTING BAD N

	 Accessed: 22-06-2016 19:41 UTC
	 Abstract match: None
	 Figure match: <regex.Match object; span=(17817, 17820), match='Fig'>
	 Figure match: <regex.Match object; span=(19449, 19452), match='Fig'>
	 Figure match: <regex.Match object; span=(49378, 49381), match='Fig'>
	 Figure match: <regex.Match object; span=(55288, 55291), match='Fig'>
	 Figure match: <regex.Match object; span=(59820, 59823), match='Fig'>
	 Figure match: <regex.Match object; span=(62259, 62262), match='Fig'>
	 Figure match: <regex.Match object; span=(65234, 65237), match='Fig'>
	 Figure match: <regex.Match object; span=(65644, 65647), match='Fig'>
	 Figure match: <regex.Match object; span=(67833, 67836), match='Fig'>
	 Reference match: <regex.Match object; span=(96029, 96039), match='REFERENCES'>
	 Section match: ['RATIONAL CHOICE', (100876, 100891)]
	 Section match: ['DETERRENCE', (100893, 100903)]
	 Section match: ['DELINQUENCY', (100909, 100920)]
	 Section match: ['FORMATION OF RISK PERCEPTIONS', (12607, 12636)]
	

In [13]:
out_ASRpre1946 = createOutputStrings(convertedStrings_ASRpre1946, folder = ASRpre1946pdfs, journal = 'ASR')
out_ASRpost1946 = createOutputStrings(convertedStrings_ASRpost1946, folder = ASRpost1946pdfs, journal = 'ASR')

writeOut(out_ASRpre1946, pdfFolder = ASRpre1946pdfs, outFolder = 'ASR_pre1946')
writeOut(out_ASRpost1946, pdfFolder = ASRpost1946pdfs, outFolder = 'ASR_post1946')

16it [00:00, 3326.17it/s]
121it [00:00, 3567.34it/s]

done!
done!
